# Using PH_EXE class

This notebook explains how to use the **PH\_EXE** class inside the **PH.ph_step_exe.vqe_step** module. 

This class, given an ansatz and its parent Hamiltonian, provided as a string  Pauli decomposition, computes the associated ground state energy (that should be near zero).

In [ ]:
import logging

In [ ]:
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)


In [ ]:
logger = logging.getLogger('__name__')

In [ ]:
import os
import sys
import numpy as np
sys.path.append("../../")

## 0. The QPU

To simulate the quantum circuits generated by the functions presented in this notebook a configured myQLM (or QLM) **Quantum Process Unit (QPU)** is mandatory. The **QPU** can execute an ideal simulation or can simulate the quantum circuits under a noisy hardware model (noisy simulation). To easily deal with these 2 kinds of simulations the  *select_qpu* function from **PH.qpu.select_qpu** was developed. The input of this function is a Python dictionary that allows to the user configure easily a **QPU**.

In the present notebook, only ideal simulation is used. Please refer to the **PH/qpu/NoisyModels.ipynb** notebook for configuring noisy models and the corresponding noisy **QPU**s

In [ ]:
sys.path.append("../../../")
from qpu.select_qpu import select_qpu

The minimum Python dictionary for configuring an ideal **QPU** is presented in the following cell. In this case, the user only has to provide a value to the *qpu_type* key. Depending on the type of simulator desired the following strings should be provided:

* *qlmass_linalg*: to use the **LinAlg Quantum Learning Machine (QLM)** algebra simulator. In this case, the computation will be sent to the **QLM** by using the  Qaptiva QLM as a Service.
* *qlmass_mps*: to use **MPS QLM** simulator. In this case, the computation will be sent to the **QLM** by using the  Qaptiva QLM as a Service.
* *python*: to use the PyLinalg algebra simulator.
* *c*: to use the CLinalg alegbra simulator.
* *linalg*: to use the **LinAlg QLM**. In this case, the user should be inside a **EVIDEN QLM**
* *mps*: to use the **MPS QLM** simulator. In this case, the user should be inside a **EVIDEN QLM**

In [ ]:
# List with the strings that should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]
qpu_config = {
    "qpu_type": ideal_qpus[0], 
}
qpu = select_qpu(qpu_config)

Before doing the siumulation we need to perform several steps:

1. Ansatz Definition and Solving (doing with functions from **ansatzes** module).
2. Parent Hamiltonian Computation (doing with functions from **parent\_hamiltonian** module)

## 1. Ansatz Definition and Solving

We neeed to define an ansatz circuit and solving it for getting its state (see **02_Ansatzes.ipynb**. We can use the functions from **PH.ansatzes.ansatzes** module

### Ansatz definition

First we define an ansatz circuit

In [ ]:
from PH.ansatzes.ansatzes import ansatz_selector, angles_ansatz01

In [ ]:
# Ansatz Configuration
ansatz = "simple01"
nqubits = 6
depth = 2
ansatz_conf = {
    'nqubits' : nqubits,
    'depth' : depth
}
filename = "ansatz_{}_nqubits_{}_depth_{}".format(
    ansatz, ansatz_conf["nqubits"], ansatz_conf["depth"])
# Create Ansatz Circuit
circuit = ansatz_selector(ansatz, **ansatz_conf)

In [ ]:
%qatdisplay circuit --svg

In [ ]:
# Fixing the parameters of the circuit
circuit, parameters_pdf = angles_ansatz01(circuit)

In [ ]:
%qatdisplay circuit --svg

In [ ]:
parameters_pdf

### Ansatz Solution

Now we need to fix the parameters of the ansatz and solving it(see notebook **02_Ansatzes.ipynb**)

In [ ]:
from PH.ansatzes.ansatzes import SolveCircuit

In [ ]:
# For saving results
from PH.utils.utils_ph import create_folder
folder = "Saving_notebook4/"
folder = create_folder(folder)

In [ ]:
# Solving  Ansatz Configuration

solve_conf = {
    "nqubits":nqubits,
    "qpu" : qpu,
    "parameters" : parameters_pdf,
    "filename": folder + filename,
    "save": True        
}
# Solving Ansatz
solv_ansatz = SolveCircuit(circuit, **solve_conf)
solv_ansatz.run()

In [ ]:
solv_ansatz.state

## 2. Parent Hamiltonian Computation

With the obtained state of the ansatz, we need to compute the Parent Hamiltonian and get its Pauli string decomposition. Here we use the **PH.parent_hamiltonian.parent_hamiltonia** module (see notebook **03_Using_PH_Class.ipynb** for more information)

In [ ]:
from PH.parent_hamiltonian.parent_hamiltonian import PH

In [ ]:
# Create PH
# Get teh amplitudes
amplitudes = list(solv_ansatz.state["Amplitude"])
ph_conf = {
    "filename": folder + filename,
    "save": True          
}
ph_object = PH(amplitudes, t_invariant=True, **ph_conf)
ph_object.local_ph()

In [ ]:
ph_object.pauli_pdf

## 4. Computing PH ground state

Now we have all mandatory inputs so we can use **PH_EXE** class from **PH.ph_step_exe.ph_step_exe** module for computing the ground state energy of the parent hamiltonian of our input ansatz.

We need to provide for initializing the class:

* *ansatz*: myqlm circuit of the ansatz where the parameters are fixed.
* *pauli_ph*: pandas DataFrame with the Pauli decomposition of the Parent Hamiltonian
* *nqubits*: number of qubits of the ansatz

Additionally, other parameters can be provided as keyword arguments (*kwargs*):
* *qpu*: myqlm QPU unit for simulating the ground state energy
* *nb_shots*: number of shots for measuring GSE.
* *t_inv*: if True for indicating that the Pauli decomposition was done using translational invariance.
* *truncation*: for truncating the number of Pauli strings. It is a positive number and all the Pauli coefficients where its absolute value is lower than $10^{-\text{truncation}}$ will be deleted

The **run** method should be used for the computation.

**BE AWARE**

If the local **PH** was computed using the translational invariance then the obtained Pauli decomposition was obtained for the first qubit only!! For executing the complete step we need to replicate the Pauli strings for all the qubits. This is done automatically by the class **BUT it is mandatory to indicate it by the t_inv key**

In [ ]:
from PH.ph_step_exe.vqe_step import PH_EXE

In [ ]:
vqe_conf = {
    "t_inv":True,
    "qpu" : qpu,
    "nb_shots": 0,
    "truncation": None,
    "filename": folder + filename,
    "save": False        
}

ansatz_circuit = solv_ansatz.circuit
pauli_ph = ph_object.pauli_pdf
nqubits = ansatz_conf["nqubits"]
exe_ph = PH_EXE(ansatz_circuit, pauli_ph, nqubits, **vqe_conf)

In [ ]:
exe_ph.run()

The results will be stored in the attribute **pdf_result** (that will be a pandas DataFrame)

In [ ]:
exe_ph.pdf_result

In [ ]:
exe_ph.pdf_info

In [ ]:
exe_ph.pdf

If save was required the complete information (axis 1 concatenation of *exe_ph.pdf_result* and *exe_ph.pdf_result*) will be saved as: folder + filename+*_phexe.csv*

## 5. run_ph_execution

Inside the module **vqe_step** the **run_ph_execution** function can be found. This function uses typical **CSV** files from **PH.ansatzes.ansatzes** and **PH.parent_hamiltonian.parent_hamiltonian** modules for executing a **VQE** step. The input of this function is a **kwargs** (called **configuration**) where following keywords are processed:

* base_fn: Base name for the files generated by **ansatzes** and **parent_hamiltonian**. The pattern of the name can be: 
    * **ansatz\_{}\_nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}**. From this kind of filenames the code extracts directly:
        * ansatz
        * nqubits
        * depth
        * The following files must exist:
            * **ansatz\_{}\_nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_parameter.csv**: with the angles of the ansatz
            * **ansatz\_{}\_nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_pauli.csv**: with the Pauli decomposition of the **PH**.
    * **nqubits\_{}\_depth\_{}\_qpu\_{}**. From this kind of filenames the code extracts directly:
        *  ansatz: the ansatz will be fixed by defalut to *simple01*
        *  nqubits
        *  depth
        * The following files must exist:
            * **nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_parameter.csv**: with the angles of the ansatz
            * **nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_pauli.csv**: with the Pauli decomposition of the **PH**.
    * Additionally filenames without the **\
    * _qpu_ansatz** can be given as valid filenames. Corresponding **_parameter.csv** and **_pauli.csv** files must exist
* qpu: with the qpu for executing the **VQE** step simulation.
* nb_shots: for setting the  number of shots for the **VQE** step.
* t_inv: True if the ansatz is transaltional invariant.
* truncation: integer for truncating the Pauli coefficients
* save: for saving the results. The results will be saved with the following pattern that depends on the *base_fn*:
    * **ansatz\_{}\_nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_ph\_exe.csv**
    * **nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_ph\_exe.csv**
    * If in the base_fn the name does not have the **\_qpu_ansatz** the created file does not have too.
    
    
We can use the files generated in this notebook for testing the **run_ph_execution** function  function    

In [ ]:
from PH.ph_step_exe.vqe_step import run_ph_execution

In [ ]:
conf = {
    "base_fn" : folder + "ansatz_simple01_nqubits_6_depth_2",
    "qpu": qpu,
    "nb_shots": 0,
    "t_inv" : True,
    "truncation" : None,
    "save": True
}

In [ ]:
pdf = run_ph_execution(**conf)

In [ ]:
pdf

## 6. Command Line execution

The **run_ph_execution** function from module **PH.ph_step_exe.vqe_step**  can be executed from the command line. Several arguments can be passed for configuring the **VQE** step computation. Help can be obtained by:

    python vqe_step.py -h

    usage: vqe_step.py [-h] [-basefn BASE_FN] [-nb_shots NB_SHOTS]
                       [-truncation TRUNCATION] [-json_qpu JSON_QPU] [-qpu_id QPU_ID]
                       [--t_inv] [--print] [--exe] [--save]

    optional arguments:
      -h, --help            show this help message and exit
      -basefn BASE_FN       Base Filename for Loading Files
      -nb_shots NB_SHOTS    Number of shots
      -truncation TRUNCATION
                            Truncation for Pauli coeficients.
      -json_qpu JSON_QPU    JSON with the qpu configuration
      -qpu_id QPU_ID        Select a QPU from a JSON file
      --t_inv               Setting translational invariant of the ansatz
      --print               For printing the selected QPU configuration.
      --exe                 For executing program
      --save                For storing results

**BE AWARE**

The configuration of the QPU should be provided as JSON file. Examples of these files are the:

* qpu_ideal.json
* qpu_noisy.json

that can be found in the **tnbs/qpu/** folder.

or the 

* qpu_for_vqe_ideal.json 

that can be found in the **PH/ph_step_exe/** folder.

These JSON files can generate several QPUs configurations with the same file so we need to select which one will be used for simulation. This can be done by providing the *-qpu_id QPU_ID* argument (in combination with the *-json_qpu JSON_QPU* argument for providing the JSON file with the QPU configuration.

**Example**

     python vqe_step.py -basefn ../Saves/ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c -json_qpu ../../../qpu/qpu_ideal.json -qpu_id 0  -truncation 3 --exe --save

In this case, we would use the *../Saves/ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c* as the base name. This implies that, previously, the following files were generated:

* Saves/ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_state.csv
* Saves/ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_parameters.csv
* Saves/ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_pauli.csv

For configuring the QPU the *../../../qpu/qpu_ideal.json* JSON file will be used and the element 0 (-qpu_id 0) of the different qpu configurations will be used for doing the simulation (by default and if the file was not changed the CLinAlg wil be used). The Pauli coefficients lower than $10^{-3}$ will be pruned (*-truncation 3* argument). The resulting DataFrame will be saved (*--save* parameter). The following file will be generated:

* Saves/ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_ph_time.csv

To get help use: 

    python launch_vqe_step.py -h. The following arguments can be provided:

    usage: launch_vqe_step.py [-h] [--count] [--all | -id ID] [--print]
                              [-json_qpu JSON_QPU] [--exe]

    optional arguments:
      -h, --help          show this help message and exit
      --count             Getting the number of elements from vqe_step.json
      --all               Select all the elements from vqe_step.json
      -id ID              Select one element from vqe_step.json
      --print             For printing the configuration.
      -json_qpu JSON_QPU  JSON with the qpu configuration
      --exe               For executing program

## 7. Massive VQE step computations

As for the case of massive ansatz state and parent Hamiltonian computations (see Notebook **02_Ansatzes.ipynb and 03_Using_PH_Class.ipynb**) we can execute massive **VQE** step computations. For this, the 2 following files can be used:

* **vqe\_step.json**: JSON file with the configuration for the **VQE** step computations. For each desired computation a complete dictionary should be provided. The keys are:
    * save: for saving or not the results
    * t_inv: for setting if the ansatz is, or not, translational invariant.
    * base_fn: base name for the inputs files.
    * nb_shots: number of shots for **VQE** step.
    * qpu_ph: qpu for executing **VQE** step.
    * truncation: for truncating Pauli coefficients.
* **launch_vqe_step.py**: This script procces the before **JSON** file creating a complete list of all posible **VQE** calculations.

To get help use: 

    usage: launch_vqe_step.py [-h] [--count] [--all | -vqe_id VQE_ID] [--print] [-json_qpu JSON_QPU] [-qpu_id QPU_ID]
                              [--exe]
    
    options:
      -h, --help          show this help message and exit
      --count             Getting the number of elements from vqe_step.json
      --all               Select all the elements from vqe_step.json
      -vqe_id VQE_ID      Select one element from vqe_step.json
      --print             For printing the configuratiÇon.
      -json_qpu JSON_QPU  JSON with the qpu configuration
      -qpu_id QPU_ID      Select which qpu to use from all possibilities
      --exe               For executing program

**NOTE**

The *-vqe_id* argument controls which of the possible **VQE** computations should be executed, while the *-qpu_id* selects which *QPU* to use from the input file provided in the **-json_qpu JSON_QPU** argument. The **--all** argument will execute all possible **VQE** computations but a single *QPU* should be selected.